In [1]:
from nltk import bigrams, word_tokenize
from nltk.util import ngrams
t = 'I am a boy'
tk = word_tokenize(t)
bg = bigrams(tk)#직전의 단어
ng = ngrams(tk,3)#특정영역
list(bg),list(ng)

([('I', 'am'), ('am', 'a'), ('a', 'boy')],
 [('I', 'am', 'a'), ('am', 'a', 'boy')])

In [2]:
#SS<문장>SE

In [3]:
data2=ngrams(tk,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
#pad_left=True 왼쪽패딩추가, left_pad_symbol="SS" 왼쪽패딩 내용
#pad_right=True 오른쪽패딩추가, right_pad_symbol="SS" 오른쪽패딩 내용
list(data2)

[('SS', 'I'), ('I', 'am'), ('am', 'a'), ('a', 'boy'), ('boy', 'SE')]

In [4]:
from nltk import ConditionalFreqDist #ConditionalFreqDist : 조건부빈도수
t = 'I am a boy'
t2 = ' you are a boy'
tk = word_tokenize(t)
ng2=ngrams(tk,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
fd = ConditionalFreqDist([(i[0],i[1]) for i in ng2])

In [5]:
fd.conditions() #정답 접근 정보

['SS', 'I', 'am', 'a', 'boy']

In [6]:
fd['SS']

FreqDist({'I': 1})

In [7]:
fd['I']

FreqDist({'am': 1})

In [8]:
fd['SE']

FreqDist({})

In [9]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
from nltk.corpus import movie_reviews
data_l = []
n=0
for i in movie_reviews.sents():
    bg2 = ngrams(i,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
    data_l +=[t for t in bg2]
    n+=1
data_l

[('SS', 'plot'),
 ('plot', ':'),
 (':', 'two'),
 ('two', 'teen'),
 ('teen', 'couples'),
 ('couples', 'go'),
 ('go', 'to'),
 ('to', 'a'),
 ('a', 'church'),
 ('church', 'party'),
 ('party', ','),
 (',', 'drink'),
 ('drink', 'and'),
 ('and', 'then'),
 ('then', 'drive'),
 ('drive', '.'),
 ('.', 'SE'),
 ('SS', 'they'),
 ('they', 'get'),
 ('get', 'into'),
 ('into', 'an'),
 ('an', 'accident'),
 ('accident', '.'),
 ('.', 'SE'),
 ('SS', 'one'),
 ('one', 'of'),
 ('of', 'the'),
 ('the', 'guys'),
 ('guys', 'dies'),
 ('dies', ','),
 (',', 'but'),
 ('but', 'his'),
 ('his', 'girlfriend'),
 ('girlfriend', 'continues'),
 ('continues', 'to'),
 ('to', 'see'),
 ('see', 'him'),
 ('him', 'in'),
 ('in', 'her'),
 ('her', 'life'),
 ('life', ','),
 (',', 'and'),
 ('and', 'has'),
 ('has', 'nightmares'),
 ('nightmares', '.'),
 ('.', 'SE'),
 ('SS', 'what'),
 ('what', "'"),
 ("'", 's'),
 ('s', 'the'),
 ('the', 'deal'),
 ('deal', '?'),
 ('?', 'SE'),
 ('SS', 'watch'),
 ('watch', 'the'),
 ('the', 'movie'),
 ('movie', 

In [11]:
len(data_l)

1655352

In [12]:
len(movie_reviews.sents()) #문장 개수

71532

In [13]:
n #시작단어의 개수

71532

In [14]:
cfd = ConditionalFreqDist(data_l)

In [15]:
cfd["SS"].most_common(10)

[('the', 8071),
 ('.', 3173),
 ('it', 3136),
 ('i', 2471),
 ('but', 1814),
 ('and', 1735),
 ('he', 1672),
 ('in', 1659),
 ('this', 1651),
 ('there', 1298)]

In [16]:
cfd["the"].most_common(10)

[('film', 4542),
 ('movie', 2147),
 ('story', 985),
 ('most', 945),
 ('first', 902),
 ('same', 774),
 ('only', 665),
 ('end', 664),
 ('best', 642),
 ('audience', 620)]

In [17]:
cfd["."].most_common(10)

[('SE', 63404), ('"', 1854), (')', 535), ("'", 70), (']', 10), ("''", 3)]

In [18]:
from nltk.probability import ConditionalProbDist, MLEProbDist # ConditionalProbDist 조건부확률
cpd = ConditionalProbDist(cfd,MLEProbDist)

In [19]:
cpd['the'].prob('movie') #the 다음에 movie가 등장할 확률

0.0280547243528597

In [20]:
cpd['movie'].prob('.')

0.13897071564720154

In [21]:
cpd['.'].prob('the')

0.0

In [22]:
import numpy as np
def s_sc_f(x): #바이그램 언어 모형의 확률 연산 함수
    p = 0.0 #확률
    for i in range(len(x)-1):
        c = x[i]
        w = x[i+1]
        p+=np.log(cpd[c].prob(w) + np.finfo(float).eps)
    return np.exp(p)

In [23]:
test_data = ['the','movie','.'] #실현가능문장
s_sc_f(test_data)

0.003898785120601922

In [24]:
test_data2 = ['movie','.','the'] #실현불가능문장
s_sc_f(test_data2)

3.085769765203191e-17

In [25]:
cpd["SS"].generate() #SS다음 시작할 단어 임의로 생성

'to'

In [26]:
import random
random.seed(10)
cpd["SS"].generate() #시드 고정으로 단어 고정

'she'

In [27]:
import random
random.seed(10)
cpd["she"].generate()

'wasn'

In [28]:
import random
random.seed(10)
type(cpd["and"].generate())

str

data = "data+=문자열".join([])

문자의 생성(조건부 확률을 기반한 바이그램 언어 모델)

In [29]:
# data = ''
# data += 문자열
# str.join()

In [30]:
st='SS'
all_str = []
while True:
    import random
    random.seed(10)
    st = cpd[st].generate()
    all_str.append(st+' ')
    if st =='SE':
        all_str.pop()
        break
''.join(all_str)

"she wasn ' s first part of these guys catch a hard 2 is . "

# 정리

In [31]:
from nltk.util import ngrams #앞단어와 뒷단어를 갖고있는 data 생성용
from nltk import ConditionalFreqDist #문맥별 단어 빈도수 측정 클래스
from nltk.probability import ConditionalProbDist #조건부 확률 추정 클래스
from nltk.probability import MLEProbDist #최대 우도 추정값 클래스
#.generate() 샘플 추출 (임의의 값을 추출)

1. 데이터 수집

In [32]:
from nltk.corpus import movie_reviews
data = movie_reviews.sents() #단어별 토큰화 된 문장들

2. 데이터 전처리
    - 토큰화
    - 정형화
    - 정규화

In [33]:
data_l = [] #처리된 단어 토큰 조합
for i in data:
    bg = ngrams(i,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
    data_l +=[t for t in bg]

3. 모델 학습

In [34]:
cfd = ConditionalFreqDist(data_l)
cpd = ConditionalProbDist(cfd,MLEProbDist)

4. 검증 (생략)

최종 동작-(data별 추가적 처리가 필요) <자동 랜덤 문장 생성기>

In [35]:
st='SS' #문장의 시작
all_str = []
import random
random.seed(10) #(seed고정으로 인한 data 고정)
while True: #SS<문장>SE
    st = cpd[st].generate() #임의로 샘플 추출 ex) i ->[am, a, data, SE]
    all_str.append(st+' ')
    if st =='SE': #문장의 종료
        all_str.pop()
        break
생성된data = ''.join(all_str) #리스트의 내용을 이용하여 하나의 문자열로 정리

In [36]:
생성된data

'she and fine effect ; frankly , and doing his son becomes shockingly lazy shortcut to her mother of " story " '